In [1]:
%%capture
import warnings
warnings.filterwarnings("ignore")

import calitp_data_analysis.magics

import pandas as pd
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS
import _new_ct_report_utils as utils
from update_vars import GTFS_DATA_DICT, analysis_month, file_name 

# Maps
import branca
import branca.colormap as cm
from shared_utils import webmap_utils
import geopandas as gpd

# Display
from great_tables import GT
from IPython.display import HTML, Image, Markdown, display, display_html
from slugify import slugify

In [2]:
import google.auth
credentials, project = google.auth.default()
import gcsfs

fs = gcsfs.GCSFileSystem()

In [3]:
from functools import cache
from calitp_data_analysis.gcs_pandas import GCSPandas

@cache
def gcs_pandas():
    return GCSPandas()

In [4]:
district = '01-Eureka'

In [5]:
# %%capture_parameters
# district

In [6]:
GCS_PATH = f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/"

In [7]:
# Files for webmaps
boundary_file = f"district_{district}_boundary"
transit_routes_file = f"district_{district}_transit_routes"
shn_file = f"district_{district}_shn"
transit_shn_file = f"district_{district}_transit_routes_shn"

In [8]:
transit_route_shs_gdf, transit_route_shs_table, district_int = (
    utils.final_transit_route_shs_outputs(15, district)
)

In [9]:
fct_monthly_routes_url = f"{GCS_PATH}{GTFS_DATA_DICT.gtfs_digest_rollup.route_map}_{file_name}.parquet"

In [10]:
operator_summary_url = f"{GCS_PATH}{GTFS_DATA_DICT.gtfs_digest_rollup.operator_summary}_{file_name}.parquet"

In [11]:
operator_df = gcs_pandas().read_parquet(
    operator_summary_url,
    filters=[
        ("Caltrans District", "==", district),
        ("Date", "==", pd.Timestamp(analysis_month)),
        ("Day Type", "==", "Weekday")
    ],
)

In [12]:
fct_monthly_route_df = gpd.read_parquet(fct_monthly_routes_url,
                                        filters=[[("Caltrans District", "==", district)]],
                                        storage_options={"token": credentials.token})

In [13]:
fct_monthly_route_df = utils.prep_gdf(fct_monthly_route_df)

In [14]:
district_gdf = utils.load_ct_district(district_int)

In [15]:
shn_gdf =  utils.load_buffered_shn_map(district_int)

# {district}
These are district summaries for [GTFS Digest](https://gtfs-digest--cal-itp-data-analyses.netlify.app/). 

## District Overview

In [16]:
try:
    operator_df2 = utils.create_summary_table(operator_df)
except:
    pass

In [17]:
try:
    display(
    GT(operator_df2
        )
        .tab_header(title=f"District {district} GTFS summary stats")
    )
except:
    pass

GT(_tbl_data=       Category  Value
0  #  Operators      1
1      #  Trips  3,479
2      #  Stops    324
3     #  Routes     11, _body=<great_tables._gt_data.Body object at 0x7bcfc84ce590>, _boxhead=Boxhead([ColInfo(var='Category', type=<ColInfoTypeEnum.default: 1>, column_label='Category', column_align='left', column_width=None), ColInfo(var='Value', type=<ColInfoTypeEnum.default: 1>, column_label='Value', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7bcfe03a4610>, _spanners=Spanners([]), _heading=Heading(title='District 01-Eureka GTFS summary stats', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x7bcfecb81a10>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, category='heading', type='value', value=None), heading_align=OptionsInfo(scss=True, category='heading', type='value', value='center'), heading_title_font_size=OptionsInfo(scss=True, category='heading', type='px', value='125%'), heading_title_font_weight=OptionsInfo(scss=True, category='heading', type='value', value='initial'), heading_subtitle_font_size=OptionsInfo(scss=True, category='heading', type='px', value='85%'), heading_subtitle_

## Routes within the District

In [18]:
color_map = cm.linear.Spectral_11.scale()

In [19]:
try:
    color_map = branca.colormap.LinearColormap(
    colors=color_map.colors[7:], vmin=0, vmax=fct_monthly_route_df.Number.max()
)
except:
    pass

In [20]:
try:
    district_map = webmap_utils.set_state_export(
    district_gdf,
    subfolder = "caltrans_district_digest/",
    filename=boundary_file,
    map_title="District Map",
    overwrite = True
)
except:
    pass

In [21]:
try:
    transit_routes = webmap_utils.set_state_export(
    fct_monthly_route_df,
    subfolder = "caltrans_district_digest/",
    filename=transit_routes_file,
    map_title="Transit Routes",
    cmap=color_map,
    color_col="Number",
    existing_state=district_map,
    overwrite = True
    
)
except:
    pass

In [22]:
try:
    webmap_utils.render_spa_link(transit_routes["spa_link"], text="Open Routes for all Operators Map")
except:
    pass

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJEaXN0cmljdCBNYXAiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL2NhbHRyYW5zX2Rpc3RyaWN0X2RpZ2VzdC9kaXN0cmljdF8wMS1FdXJla2FfYm91bmRhcnkuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX0sIHsibmFtZSI6ICJUcmFuc2l0IFJvdXRlcyIsICJ1cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2NhbGl0cC1tYXAtdGlsZXMvY2FsdHJhbnNfZGlzdHJpY3RfZGlnZXN0L2Rpc3RyaWN0XzAxLUV1cmVrYV90cmFuc2l0X3JvdXRlcy5nZW9qc29uLmd6IiwgInByb3BlcnRpZXMiOiB7InN0cm9rZWQiOiBmYWxzZSwgImhpZ2hsaWdodF9zYXR1cmF0aW9uX211bHRpcGxpZXIiOiAwLjV9fV0sICJsYXRfbG9uIjogWzQwLjI0MTY2Nzk0ODcxNzQ4LCAtMTIzLjU0ODc1ODA4MjM1NTYzXSwgInpvb20iOiAxM30=" target="_blank">Open Open Routes for all Operators Map in New Tab</a>

In [23]:
try:
    webmap_utils.display_spa_map(transit_routes["spa_link"])
except:
    pass

## Transit Routes on the State Highway Network
**Only transit routes that have 15% or more if its length on one or more State Highway Network routes are included**

In [24]:
color_map2 = cm.linear.RdYlBu_11.scale()

In [25]:
color_map2 = branca.colormap.LinearColormap(
    colors=color_map2.colors[7:], vmin=0, vmax=100
)

In [26]:
try:
    shn_map = webmap_utils.set_state_export(
    shn_gdf,
    subfolder = "caltrans_district_digest/",
    filename=shn_file,
    map_title="State Highway Network Map",
    map_type='state_highway_network',
    overwrite = True
)
except:
    pass

In [27]:
try:
    transit_shn_map = webmap_utils.set_state_export(
    transit_route_shs_gdf,
    subfolder = "caltrans_district_digest/",
    filename=transit_shn_file,
    map_title="Transit Routes on the State Highway Network",
    cmap=color_map2,
    color_col="Percentage of Transit Route on SHN Across All Districts",
    existing_state=shn_map,
    legend_url="https://storage.googleapis.com/calitp-map-tiles/transit_route_pct.svg",
    overwrite = True
    
)
except:
    pass

In [28]:
try:
    webmap_utils.render_spa_link(transit_shn_map["spa_link"], text="Open Routes on State Highway System Map")
except:
    pass

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTdGF0ZSBIaWdod2F5IE5ldHdvcmsgTWFwIiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy9jYWx0cmFuc19kaXN0cmljdF9kaWdlc3QvZGlzdHJpY3RfMDEtRXVyZWthX3Nobi5nZW9qc29uLmd6IiwgInByb3BlcnRpZXMiOiB7InN0cm9rZWQiOiBmYWxzZSwgImhpZ2hsaWdodF9zYXR1cmF0aW9uX211bHRpcGxpZXIiOiAwLjV9LCAidHlwZSI6ICJzdGF0ZV9oaWdod2F5X25ldHdvcmsifSwgeyJuYW1lIjogIlRyYW5zaXQgUm91dGVzIG9uIHRoZSBTdGF0ZSBIaWdod2F5IE5ldHdvcmsiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL2NhbHRyYW5zX2Rpc3RyaWN0X2RpZ2VzdC9kaXN0cmljdF8wMS1FdXJla2FfdHJhbnNpdF9yb3V0ZXNfc2huLmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNX19XSwgImxhdF9sb24iOiBbNDAuMTg1MzQ5OTYzMDMzMDEsIC0xMjMuNTQzMTQyODMxNDExNzddLCAiem9vbSI6IDEzLCAibGVnZW5kX3VybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy90cmFuc2l0X3JvdXRlX3BjdC5zdmcifQ==" target="_blank">Open Open Routes on State Highway System Map in New Tab</a>

In [29]:
try:
    webmap_utils.display_spa_map(transit_shn_map["spa_link"])
except:
    pass

In [30]:
try:
    display(GT(
    transit_route_shs_table.sort_values(
        by=[
            "Analysis Name",
            "Percentage of Transit Route on SHN Across All Districts",
        ],
        ascending=[True, False],
    )
))
except:
    pass

Analysis Name,Route,State Highway Network Routes in District 01-Eureka,Percentage of Transit Route on SHN Across All Districts
Humboldt Transit Authority,North State Express: Route 101,"211, 1, 101, 20, 254, 271, 283, 162, 36",97.1
Humboldt Transit Authority,North State Express: 299,"200, 96, 101, 299",90.6
Humboldt Transit Authority,Southern Humboldt Line,"36, 211, 101, 283, 254",79.9
Humboldt Transit Authority,Redwood Transit System,"211, 101, 255, 283, 299, 36, 200",54.2
Humboldt Transit Authority,Eureka Gold Route,101,15.1
Lake Transit Authority,"3 Highway 29, Clearlake to Deer Park","53, 175, 29",88.6
Lake Transit Authority,"1 Northshore, Clearlake to Lakeport","53, 29, 20",85.6
Lake Transit Authority,"2 Highway 175, Kit’s Corner to Middletown","281, 175, 29",84.3
Lake Transit Authority,7 Lakeport – Ukiah,"101, 29, 20",75.6
Lake Transit Authority,"4 Southshore, Clearlake to Lakeport","281, 53, 175, 29",75.1


## GTFS Stats by Operator

In [31]:
try:
    gtfs_table = utils.create_operator_table(operator_df)
except:
    pass

In [32]:
try:
    display((
    GT(gtfs_table.sort_values("Daily Trips", ascending=False))
    .fmt_integer(
        columns=[
            c
            for c in gtfs_table.columns
            if c not in ["Operator"]
        ]
    )
    .tab_header(
        title=f"District {district}",
        subtitle="Daily GTFS schedule statistics for Weekday by operator",
    )
    .cols_align(
        columns=[
            c
            for c in gtfs_table.columns
        ],
        align="center",
    )
))
except:
    pass

GT(_tbl_data=                     Operator  Daily Trips   # Routes   # Shapes     # Stops
0  Humboldt Transit Authority        158.1  11.227273  56.954545  324.590909, _body=<great_tables._gt_data.Body object at 0x7bcfc8149a50>, _boxhead=Boxhead([ColInfo(var='Operator', type=<ColInfoTypeEnum.default: 1>, column_label='Operator', column_align='center', column_width=None), ColInfo(var='Daily Trips', type=<ColInfoTypeEnum.default: 1>, column_label='Daily Trips', column_align='center', column_width=None), ColInfo(var='# Routes', type=<ColInfoTypeEnum.default: 1>, column_label='# Routes', column_align='center', column_width=None), ColInfo(var='# Shapes', type=<ColInfoTypeEnum.default: 1>, column_label='# Shapes', column_align='center', column_width=None), ColInfo(var='# Stops', type=<ColInfoTypeEnum.default: 1>, column_label='# Stops', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7bcfc81493d0>, _spanners=Spanners([]), _heading=Heading(title='District 01-Eureka', subtitle='Daily GTFS schedule statistics for Weekday by operator', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x7bcfecb81a10>, _formats=[<great_tables._gt_data.FormatInfo object at 0x7bcfc8130b50>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='tabl